In [287]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.download('BTC-USD')
btc_df = btc_df.reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Open','High','Low','Adj Close', 'Volume', 'Unnamed: 0', "wallets", "address", "mined"])

[*********************100%***********************]  1 of 1 completed


In [288]:
df['change'] = (df['price'].pct_change())*100
df = df.dropna()
df

,date,price,change
1,2010-09-26,0.100000,0.000000
2,2010-09-27,0.100000,0.000000
3,2010-09-28,0.100000,0.000000
4,2010-09-29,0.100000,0.000000
5,2010-09-30,0.100000,0.000000
...,...,...,...
4420,2022-11-29,16444.982422,1.403809
4421,2022-11-30,17168.566406,4.400029
4422,2022-12-01,16967.132812,-1.173270
4423,2022-12-02,17088.660156,0.716252


In [289]:
df["month"] = df['date'].map(lambda x: x.strftime('%B'))
df["year"] = df['date'].map(lambda x: x.year)
df.dtypes

date      datetime64[ns]
price            float64
change           float64
month             object
year               int64
dtype: object

In [293]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "delta",
    value =  (df.price.iloc[-1]),
    domain = {'row': 1, 'column': 1}))
fig.add_trace(go.Indicator(
    mode = "delta",
    value =  (df.price.iloc[-1]) - (df.price.iloc[-2]),
    domain = {'row': 1, 'column': 2}))

fig.update_layout(
    grid = {'rows': 2, 'columns': 2, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Current Price"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 90}}]
                         }})